<h1>MongoDB and JSON: Using Python to Insert Data</h1>
Data is pulled from the google maps api and inserted into MongoDB

In [1]:
#Dependencies

##create tables
import pandas as pd 

##Tool for converting Zip Codes into Latitude and Longitude coordinates
import pgeocode 

##Use API key
from config import gkey 

##API lookup
import requests

##Reading JSONS
import json 

##Connecting and using MongoDB commands in Python
import pymongo 

##Create artificial delay if lookups cause timeouts
import time 

In [2]:
#Connect to mongo db and create "locations_mdb" database
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

#Create "locations_mdb" database and assign it to a variable
#(note: MongoDB automatically creates a database when you call it, so this will do both)
db = client.locations_mdb

In [3]:
#Build the zip code reference table in PANDAS
zips_df=pd.read_csv('resources/sf_zips.csv')
zips_df.head()

NameError: name 'zips_df' is not defined

In [17]:


#Use the pgeocode tool to convert zips to latitude and longitude, add to table.
tool = pgeocode.Nominatim('US')

for index, row in zips_df.iterrows():
    zipcode = row['ZIPCode']
    tool_table=tool.query_postal_code(zipcode)
    lat=tool_table["latitude"]
    long=tool_table["longitude"]
    zips_df.loc[index, 'Latitude']=lat
    zips_df.loc[index, 'Longitude']=long
    

In [13]:
#Check new table
zips_df.head()

,Unnamed: 0,ZIPCode,Type,County,Population,Area Code(s),Latitude,Longitude
0,0,94102,Standard,San Francisco,"31,176",415 / 510,37.7813,-122.417
1,1,94103,Standard,San Francisco,"27,170",415 / 510 / 650,37.7725,-122.415
2,2,94104,Standard,San Francisco,406,415 / 510 / 650 / 628,37.7915,-122.402
3,3,94105,Standard,San Francisco,"5,846",415 / 510 / 650 / 628,37.7864,-122.389
4,4,94107,Standard,San Francisco,"26,599",415 / 510 / 650,37.7621,-122.397


In [4]:
#HOTELS collection: build from google api data
#load into mongoDB

api_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in zips_df.iterrows():
    time.sleep(0.5)
    target_coordinates = f"{zips_df.loc[index, 'Latitude']},{zips_df.loc[index, 'Longitude']}" 
    target_radius = 50000
    target_type = "lodging"

# set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": gkey}

# run a request using our params dictionary
    response = requests.get(api_url, params=params).json()
    
    
    db.hotels.insert_one(
        {
            'zip_code': row['ZIPCode'],
            'type': row['Type'],
            'county': row['County'],
            'population': row['Population'],
            'area_codes': row['Area Code(s)'],
            'latitude': row['Latitude'],
            'longitude': row['Longitude'],
            'hotels': response["results"]
        }
    )

In [39]:
#Build hotels database in PANDAS
hotels_df = pd.DataFrame(columns = ['Name', 'Address', "Status", 'Lattitude', "Longitude"]) 

api_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in zips_df.iterrows():
    time.sleep(0.5)
    target_coordinates = f"{zips_df.loc[index, 'Latitude']},{zips_df.loc[index, 'Longitude']}" 
    target_radius = 50000
    target_type = "lodging"

# set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": gkey}

    response = requests.get(api_url, params=params).json()

# create a pandas dataframe of hotels
    for i in range(0,len(response["results"])):
        name=response["results"][i]["name"]
        lat=response["results"][i]["geometry"]["location"]["lat"]
        long=response["results"][i]["geometry"]["location"]["lng"]
        status=response["results"][i]["business_status"]
        address=response["results"][i]['vicinity']

        hotels_df = hotels_df.append({'Name' : name, 'Address' : address, 'Status' : status, "Latitude" :lat, "Longitude": long},  
                ignore_index = True) 
    

In [57]:
#Drop any duplicates
hotels_df.drop_duplicates(keep='first', inplace=True, ignore_index=True)

In [56]:
    #Insert entries into MongoDB
    for index, row in hotels_df.iterrows():

        db.hotels.insert_one(
            {
                'Name': f"{hotels_df.loc[index, 'Name']}",
                'Address': f"{hotels_df.loc[index, 'Address']}",
                'Status': f"{hotels_df.loc[index, 'Status']}",
                'Lattitude': f"{hotels_df.loc[index, 'Latitude']}",
                'Longitude': f"{hotels_df.loc[index, 'Longitude']}"

            }
        )

In [ ]:
#Retrieve your data